In [106]:
!gcloud services enable ml.googleapis.com
!gcloud services enable compute.googleapis.com

In [107]:
!mkdir friday_training

mkdir: cannot create directory ‘friday_training’: File exists


In [ ]:
!touch ./friday_training/__init__.py

In [1]:
%%writefile ./friday_training/train.py
import datetime
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import subprocess
from google.cloud import storage
import pandas as pd
import numpy as np

from sklearn import preprocessing
from math import sqrt
import datetime
#import matplotlib.pyplot as plt
#import seaborn as sns
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve

# Fill in your Cloud Storage bucket name
BUCKET_ID = 'friday_demo2'

public_bucket = storage.Client().bucket(BUCKET_ID)
blob = public_bucket.blob('Data/train.csv')
blob.download_to_filename('train.csv')

blob = public_bucket.blob('Data/test.csv')
blob.download_to_filename('test.csv')

#Read the data from the bucket
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


numeric_features = train.select_dtypes(include=[np.number])
numeric_features.dtypes


# categorical columns
categorical_columns = ["Product_ID", "Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years",
                       "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]


# Join Train and Test Dataset so it can be cleaned all at once
train['source']='train'
test['source']='test'

data = pd.concat([train,test], ignore_index = True, sort = False)

#imput the value -2 for NaN since NaN most likely means that the person did not buy products from these categories
data["Product_Category_2"]= \
data["Product_Category_2"].fillna(-2.0).astype("float")

data["Product_Category_3"]= \
data["Product_Category_3"].fillna(-2.0).astype("float")

#Get index of all columns with product_category_1 equal 19 or 20 from train and remove since not populated
condition = data.index[(data.Product_Category_1.isin([19,20])) & (data.source == "train")]
data = data.drop(condition)


# convert categorical data to to numerical values.
# convert data in categorical columns to numerical values
encoders = {col:LabelEncoder() for col in categorical_columns}
for col in categorical_columns:
    data[col] = encoders[col].fit_transform(data[col])

#create count features of each user
def getCountVar(compute_df, count_df, var_name):
    grouped_df = count_df.groupby(var_name)
    count_dict = {}
    for name, group in grouped_df:
        count_dict[name] = group.shape[0]

    count_list = []
    for index, row in compute_df.iterrows():
        name = row[var_name]
        count_list.append(count_dict.get(name, 0))
    return count_list

data["Age_Count"]  =getCountVar(data, data, "Age")
data["Occupation_Count"]  =getCountVar(data, data, "Occupation")
data["Product_Category_1_Count"]  =getCountVar(data, data,"Product_Category_1")
data["Product_Category_2_Count"]  =getCountVar(data, data, "Product_Category_2")
data["Product_Category_3_Count"]  =getCountVar(data, data,"Product_Category_3")
data["Product_ID_Count"]  =getCountVar(data, data, "Product_ID")

#Divide into test and train
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
test.drop(['source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

# remove column we are trying to predict ('Purchase') from features list
train_features = train.drop('Purchase', axis=1)
test_features = test.drop('Purchase', axis=1)
# create training labels list
train_labels = (train['Purchase'])
test_labels = test['Purchase']

# load data into DMatrix object
dtrain = xgb.DMatrix(train_features, train_labels)
dtest = xgb.DMatrix(test_features)
# train model
bst = xgb.train({}, dtrain, 20)


# Export the model to a file
model = 'model.bst'
bst.save_model('./model.bst')

# Upload the model to Cloud Storage
bucket = storage.Client().bucket(BUCKET_ID)
blob = bucket.blob(model)
blob.upload_from_filename(model)





/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/jupyter/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/jupyter/.local/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [2]:
bst.fit(train_features, train_labels)
y_pred = bst.predict(dtest)
acc = accuracy_score(test_labels, y_pred.round())
print(acc, '\n')

AttributeError: 'Booster' object has no attribute 'fit'

In [ ]:
y_pred = model.predict(x_test)
acc = accuracy_score(y_test, y_pred.round())
print(acc, '\n')

In [81]:
import time

# Define a timestamped job name
JOB_NAME = "friday_training_{}".format(int(time.time()))
BUCKET_NAME = 'friday_demo2'

In [82]:

# Submit the training job:
!gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir gs://$BUCKET_NAME/friday_job_dir \
  --package-path ./friday_training \
  --module-name friday_training.train \
  --region us-east1 \
  --runtime-version=1.12 \
  --python-version=3.5 \
  --scale-tier BASIC \
  --stream-logs \
  -- \
  --bucket-name $BUCKET_NAME

Job [friday_training_1564604710] submitted successfully.
INFO	2019-07-31 20:25:15 +0000	service		Validating job requirements...
INFO	2019-07-31 20:25:18 +0000	service		Waiting for training program to start.
INFO	2019-07-31 20:26:34 +0000	master-replica-0		Running task with arguments: --cluster={"master": ["127.0.0.1:2222"]} --task={"type": "master", "index": 0} --job={  "package_uris": ["gs://friday_demo2/friday_job_dir/packages/1954ba646e37bebd919cb50e2e5f2c36fcba8f4c7a713346c38b2c477223f738/friday_training-0.0.0.tar.gz"],  "python_module": "friday_training.train",  "args": ["--bucket-name", "friday_demo2"],  "region": "us-east1",  "runtime_version": "1.12",  "job_dir": "gs://friday_demo2/friday_job_dir",  "run_on_raw_vm": true,  "python_version": "3.5"}
INFO	2019-07-31 20:27:30 +0000	master-replica-0		Running module friday_training.train.
INFO	2019-07-31 20:27:30 +0000	master-replica-0		Downloading the package: gs://friday_demo2/friday_job_dir/packages/1954ba646e37bebd919cb50e2e5f2c3

In [83]:
MODEL_NAME = "BlackFridayPredictor"
VERSION_NAME = "friday_predictor"
#VERSION_NAME = "friday_predictor_{}".format(int(time.time()))

In [42]:
!gcloud ml-engine models create $MODEL_NAME --regions us-east1

ERROR: (gcloud.ml-engine.models.create) Resource in project [ml-sandbox-1-191918] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


In [29]:
!gcloud ml-engine versions create $VERSION_NAME \
  --model=$MODEL_NAME \
  --framework=xgboost \
  --origin=gs://$BUCKET_NAME/ \
  --python-version=3.5 \
  --runtime-version=1.12

Creating version (this might take a few minutes)......done.


In [42]:
import json
b = test_features_10.values.tolist()
json_file = "data.json"

with open(json_file, 'w', encoding='utf-8') as f:
    for i in b:
        instance = {"input": i}
        json.dump(instance, f, sort_keys=True)
        f.write("\n")

In [8]:
test_features_10 = test_features.head(10)
test_features_10

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Age_Count,Occupation_Count,Product_Category_1_Count,Product_Category_2_Count,Product_Category_3_Count,Product_ID_Count
550068,1000004,1216,1,4,7,1,2,1,0,10,0,64902,83667,200699,20230,541656,1333
550069,1000009,1063,1,2,17,2,0,0,2,4,0,311554,57076,28791,37165,541656,371
550070,1000010,2799,0,3,1,1,4,1,4,13,0,155898,67329,215950,78834,541656,217
550071,1000010,1379,0,3,1,1,4,1,3,8,0,155898,67329,16756,8177,541656,23
550072,1000011,535,0,2,1,2,1,0,3,4,9,311554,67329,16756,37165,13115,781
550073,1000013,3407,1,4,1,2,3,1,1,2,12,64902,67329,34056,4123,39968,413
550074,1000013,1480,1,4,1,2,3,1,0,10,12,64902,67329,200699,20230,39968,602
550075,1000013,3617,1,4,1,2,3,1,1,3,6,64902,67329,34056,36705,16532,242
550076,1000015,1543,1,2,7,0,1,0,9,12,13,311554,83667,7373,15054,46469,166
550077,1000022,673,1,1,15,0,4,0,4,13,0,141209,17257,215950,78834,541656,523


In [28]:
import json
test_feat = test_features_10.values.tolist()
#export = test_features_10.json('data.json', orient='record')
json_data = json.dumps(test_feat)

In [86]:
export = test_features_10.json('data.json', orient='record')

AttributeError: 'DataFrame' object has no attribute 'json'

In [56]:
DATA_FORMAT="text" # JSON data format
INPUT_PATHS='data.json'
OUTPUT_PATH='gs://$BUCKET_NAME/'
MODEL_NAME = "BlackFridayPredictor"
VERSION_NAME = "friday_predictor"
REGION='us-east1'
#now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME = "friday_training_{}".format(int(time.time()))
MAX_WORKER_COUNT="20"

In [77]:
INPUT_FILE="data.json"

!gcloud ai-platform predict --model $MODEL_NAME --version \
  $VERSION_NAME --json-instances $INPUT_FILE



[16196.408203125, 10362.009765625]


In [76]:
%%writefile $INPUT_FILE
[1000004,1216,1,4,7,1,2,1,0,10,0,64902,83667,200699,20230,541656,1333]
[1000009,1063,1,2,17,2,0,0,2,4,0,311554,57076,28791,37165,541656,371]

Overwriting data.json


In [ ]:
%%writefile $INPUT_FILE
["User_ID",1000004,"Product_ID",1216,"Gender",1,"Age",4,"Occupation",7,"City_Category",1,"Stay_In_Current_City_Years",2,"Marital_Status",1,"Product_Category_1",0,"Product_Category_2",10,"Product_Category_3",0,"Age_Count",64902,"Occupation_Count",83667,"Product_Category_1_Count",200699,"Product_Category_2_Count",20230,"Product_Category_3_Count",541656,"Product_ID_Count",1333]
["User_ID",1000009,"Product_ID",1063,"Gender",1,"Age",2,"Occupation",17,"City_Category",2,"Stay_In_Current_City_Years",0,"Marital_Status",0,"Product_Category_1",2,"Product_Category_2",4,"Product_Category_3",0,"Age_Count",311554,"Occupation_Count",57076,"Product_Category_1_Count",28791,"Product_Category_2_Count",37165,"Product_Category_3_Count",541656,"Product_ID_Count",371]

In [89]:
!pip install witwidget --quiet

In [90]:
import witwidget

ImportError: No module named 'witwidget'